In [1]:
# Importar las librerías necesarias
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np


In [2]:
# Hiperparámetros
batch_size = 64
learning_rate = 0.001
num_epochs = 20
patience = 3  # Paciencia para el Early Stopping


In [3]:
# Transformaciones para normalizar los datos
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])


In [4]:
# Descargar y cargar el dataset MNIST
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Dividir el conjunto de entrenamiento en entrenamiento y validación
train_dataset, validation_dataset = torch.utils.data.random_split(train_dataset, [50000, 10000])

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [9]:
# Definir el modelo de red neuronal
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
        
    def forward(self, x):
        return self.model(x)

model = NeuralNet()


In [10]:
# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [11]:
# Implementación del Early Stopping
best_loss = np.Inf
patience_counter = 0

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        # Adelante
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Atrás
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Validación
    model.eval()
    validation_loss = 0.0
    with torch.no_grad():
        for images, labels in validation_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            validation_loss += loss.item() * images.size(0)
    validation_loss = validation_loss / len(validation_loader.dataset)
    
    print(f'Epoch {epoch+1}, Validation Loss: {validation_loss}')
    
    # Comprobar si el Early Stopping debe activarse
    if validation_loss < best_loss:
        best_loss = validation_loss
        patience_counter = 0
        # Guardar el mejor modelo
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print('¡Deteniendo temprano el entrenamiento!')
            break


Epoch 1, Validation Loss: 0.16777165462970733
Epoch 2, Validation Loss: 0.12191599147319794
Epoch 3, Validation Loss: 0.10976265812814236
Epoch 4, Validation Loss: 0.09811249305605889
Epoch 5, Validation Loss: 0.0954665941953659
Epoch 6, Validation Loss: 0.09319879391491413
Epoch 7, Validation Loss: 0.0986117089599371
Epoch 8, Validation Loss: 0.11268075958192349
Epoch 9, Validation Loss: 0.10372085772790014
¡Deteniendo temprano el entrenamiento!


In [13]:
# Cargar el mejor modelo guardado
model.load_state_dict(torch.load('best_model.pt'))

# Prueba del modelo
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Precisión en el conjunto de prueba: {100 * correct / total}%')


/var/folders/n0/d8kx4bjx6nlb3w_y_c7670rh0000gn/T/ipykernel_66705/1633850622.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.

Precisión en el conjunto de prueba: 97.32%
